In [ ]:
# !git clone https://github.com/oscarcapote/MMSBM_library

Cloning into 'MMSBM_library'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 318 (delta 20), reused 34 (delta 14), pack-reused 269
Receiving objects: 100% (318/318), 322.41 KiB | 3.93 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
# !rm -r MMSBM_library/

In [6]:
import pandas as pd
import numpy as np
from numba import jit
import sys, os




sys.path.append(r'..')
import MMSBM_library as sbm
from MMSBM_library.functions.utils import save_MMSBM_parameters,add_codes,load_EM_parameters

In [7]:
!pwd

/home/oscarf/Documents/MMSBM_library


In [8]:
# Download MovieLens data.
# print("Downloading movielens data...")
# from urllib.request import urlretrieve
# import zipfile

# urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
# zip_ref = zipfile.ZipFile('movielens.zip', "r")
# zip_ref.extractall()
# print("Done. Dataset contains:")
# print(zip_ref.read('ml-100k/u.info'))

In [9]:
genres = ["unknown",
        "Action",
        "Adventure",
        "Animation",
        "Children's",
        "Comedy",
        "Crime",
        "Documentary",
        "Drama",
        "Fantasy",
        "Film-Noir",
        "Horror",
        "Musical",
        "Mystery",
        "Romance",
        "Sci-Fi",
        "Thriller",
        "War",
        "Western"]

In [10]:
df_users = pd.read_csv("ml-100k/u.user",sep="|",names=["uid","age","gender","occupation","zip_code"])
df_movies = pd.read_csv("ml-100k/u.item",sep="|",usecols=[0,1,2],names=["mid","title","date"], encoding='latin-1')


In [11]:
df_movies_genres =  pd.read_csv("ml-100k/u.item",sep="|",usecols=[0]+[i for i in range(5,24)],names =["mid"]+ genres, encoding='latin-1')

L = []

for i in df_movies_genres.iloc():
    L.append( "|".join([g for g in genres if i[g]!=0]))
df_movies["genres"] = L

In [12]:
df_movies

,mid,title,date,genres
0,1,Toy Story (1995),01-Jan-1995,Animation|Children's|Comedy
1,2,GoldenEye (1995),01-Jan-1995,Action|Adventure|Thriller
2,3,Four Rooms (1995),01-Jan-1995,Thriller
3,4,Get Shorty (1995),01-Jan-1995,Action|Comedy|Drama
4,5,Copycat (1995),01-Jan-1995,Crime|Drama|Thriller
...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,Drama
1678,1679,B. Monkey (1998),06-Feb-1998,Romance|Thriller
1679,1680,Sliding Doors (1998),01-Jan-1998,Drama|Romance
1680,1681,You So Crazy (1994),01-Jan-1994,Comedy


In [13]:
users = sbm.nodes_layer(nodes_info=df_users,K=10,nodes_name="uid")
users.add_exclusive_metadata(10,"occupation")

movies = sbm.nodes_layer(nodes_info=df_movies,K=10,nodes_name="mid")
movies.add_inclusive_metadata(123,"genres",10,separator="|")



In [14]:
m = movies["genres"]

In [15]:
movies.df[str(m)+"_id"] = movies.df[str(m)]
for g in m.dict_codes:
    movies.df[str(m)+"_id"] = movies.df[str(m)+"_id"].str.replace(g,str(m.dict_codes[g]))

In [16]:
# links and neighbours
meta_name = "genres"
separator = "|"
df_dropna = movies.df.dropna(subset=[meta_name])
meta_list = movies.df[meta_name].values

observed = df_dropna[movies.node_type].values  # Nodes with known metadata
observed_id = df_dropna[movies.node_type + "_id"].values  # Nodes with known metadata


In [17]:
observed_id

array([   0,    1,    2, ..., 1679, 1680, 1681], dtype=int16)

In [18]:
for f in range(1,6):
  os.mkdir(f"./kf_{f}" )

FileExistsError: [Errno 17] File exists: './kf_1'

In [23]:
for f in range(1,6):

    df_links = pd.read_csv("ml-100k/u{}.base".format(f),sep="\t",names=["uid","mid","rating","time_stamp"])
    ratings = sbm.BiNet(df_links,"rating",nodes_a=users,nodes_b=movies)

    ratings.init_EM()

    for step in range(1):
        # print(step)
        ratings.EM_step()

        #     if step% 10==0:
    print("saving!")
    save_MMSBM_parameters(ratings,dir="./kf_{}".format(f),BiNet_json=True)


saving!
saving!
saving!
saving!
saving!


In [11]:
sbm.BiNet.load_BiNet_from_json("./kf_1/BiNet_data.json",df_links,"ratings")

NameError: name 'sbm' is not defined

In [12]:
!ls kf_1

BiNet_data.json  qka_occupation.npy  theta_a.npy  zeta_genres.npy
pkl.npy		 q_k_tau_genres.npy  theta_b.npy


In [13]:
movies.theta

NameError: name 'movies' is not defined

In [ ]:
sbm.functions.utils.save_nodes_layer_dict(users)

In [ ]:
sbm.functions.utils.save_nodes_layer_dict(movies)

In [ ]:
!ls -ltr

In [17]:
help(sbm.nodes_layer.__init__)

Help on function __init__ in module MMSBM_library:

__init__(self, K, nodes_name, nodes_info, *, separator='\t', dict_codes=None, **kwargs)
    Initialization of the nodes_layer class
    
    Parameters
    ----------
    K : int
        Number of memberships groups for the layer.
    nodes_name : str
        Name of the nodes column in the nodes_layer class
    nodes_info : str or pandas DataFrame
        If it is a string, it is the directory of the file with the nodes information.
        If it is a DataFrame, it is the DataFrame with the nodes information.
    separator : str, default        
        Separator if the columns of the file that contains the nodes information
    dict_codes : dict, default None
        Dictionary with the integer id of the nodes. The key is the nodes' name and the value its id.



In [18]:
import json
with open("./uid_data.json", "r") as f:
  data = json.load(f)

In [19]:
df_users = pd.read_csv("ml-100k/u.user",sep="|",names=["uid","age","gender","occupation","zip_code"])
# df_movies = pd.read_csv("ml-100k/u.item",sep="|",usecols=[0,1,2],names=["mid","title","date"], encoding='latin-1')


In [20]:
users2 = sbm.nodes_layer.load_nodes_layer_from_file(df_users,"./uid_data.json")

In [21]:
df_users

,uid,age,gender,occupation,zip_code,uid_id,occupation_id
0,1,24,M,technician,85711,0,19
1,2,53,F,other,94043,1,13
2,3,23,M,writer,32067,2,20
3,4,24,M,technician,43537,3,19
4,5,33,F,other,15213,4,13
...,...,...,...,...,...,...,...
938,939,26,F,student,33319,938,18
939,940,32,M,administrator,02215,939,0
940,941,20,M,student,97229,940,18
941,942,48,F,librarian,78209,941,10


In [22]:
users.df

,uid,age,gender,occupation,zip_code,uid_id,occupation_id
0,1,24,M,technician,85711,0,19
1,2,53,F,other,94043,1,13
2,3,23,M,writer,32067,2,20
3,4,24,M,technician,43537,3,19
4,5,33,F,other,15213,4,13
...,...,...,...,...,...,...,...
938,939,26,F,student,33319,938,18
939,940,32,M,administrator,02215,939,0
940,941,20,M,student,97229,940,18
941,942,48,F,librarian,78209,941,10


In [23]:
np.all(users2.df["occupation_id"] == users.df["occupation_id"])

True

In [24]:
data2.keys()

NameError: name 'data2' is not defined

In [25]:
for m in data["metadata_exclusives"]:
    print(m)
    users2.add_exclusive_metadata(**m)

{'meta_name': 'occupation', 'lambda_val': 10, 'N_atts': 21, 'dict_codes': {'technician': '19', 'other': '13', 'writer': '20', 'executive': '6', 'administrator': '0', 'student': '18', 'lawyer': '9', 'educator': '3', 'scientist': '17', 'entertainment': '5', 'programmer': '14', 'librarian': '10', 'homemaker': '8', 'artist': '1', 'engineer': '4', 'marketing': '11', 'none': '12', 'healthcare': '7', 'retired': '15', 'salesman': '16', 'doctor': '2'}}


In [26]:
df_movies2 =  df_movies#.drop("genres_id", axis='columns')


In [27]:
movies2 = sbm.nodes_layer.load_nodes_layer_from_file(df_movies2,"./mid_data.json")

In [28]:
import json
with open("./mid_data.json", "r") as f:
  data2 = json.load(f)

In [29]:
for m in data2["metadata_inclusives"]:
    movies2.add_inclusive_metadata(**m)

In [30]:
movies2.df

,mid,title,date,genres,mid_id,genres_id
0,1,Toy Story (1995),01-Jan-1995,Animation|Children's|Comedy,0,0|1|2
1,2,GoldenEye (1995),01-Jan-1995,Action|Adventure|Thriller,1,3|4|5
2,3,Four Rooms (1995),01-Jan-1995,Thriller,2,5
3,4,Get Shorty (1995),01-Jan-1995,Action|Comedy|Drama,3,3|2|6
4,5,Copycat (1995),01-Jan-1995,Crime|Drama|Thriller,4,7|6|5
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,Drama,1677,6
1678,1679,B. Monkey (1998),06-Feb-1998,Romance|Thriller,1678,10|5
1679,1680,Sliding Doors (1998),01-Jan-1998,Drama|Romance,1679,6|10
1680,1681,You So Crazy (1994),01-Jan-1994,Comedy,1680,2


In [31]:
movies.df

,mid,title,date,genres,mid_id,genres_id
0,1,Toy Story (1995),01-Jan-1995,Animation|Children's|Comedy,0,0|1|2
1,2,GoldenEye (1995),01-Jan-1995,Action|Adventure|Thriller,1,3|4|5
2,3,Four Rooms (1995),01-Jan-1995,Thriller,2,5
3,4,Get Shorty (1995),01-Jan-1995,Action|Comedy|Drama,3,3|2|6
4,5,Copycat (1995),01-Jan-1995,Crime|Drama|Thriller,4,7|6|5
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,Drama,1677,6
1678,1679,B. Monkey (1998),06-Feb-1998,Romance|Thriller,1678,10|5
1679,1680,Sliding Doors (1998),01-Jan-1998,Drama|Romance,1679,6|10
1680,1681,You So Crazy (1994),01-Jan-1994,Comedy,1680,2


In [32]:
meta = users2["occupation"]

In [33]:
meta.qka

AttributeError: 'exclusive_metadata' object has no attribute '_qka'

In [10]:
sbm.BiNet.load_BiNet_from_json("./kf_1/BiNet_data.json", df_links, "votes")

KeyError: 1325

In [11]:
import json
with open("./kf_1/BiNet_data.json", "r") as f:
  data = json.load(f)

In [ ]:
sbm.nodes_layer.create_simple_layer(Kb, self.df[nodes_b_name].unique(), nodes_b_name, dict_codes = dict_codes_b)

In [ ]:
users.df

In [ ]:
ratings.links_training.max(axis=0)

In [ ]:
def omega_comp_arrays(Na,Nb,p_kl,theta,eta,K,L,links_array,links_ratings):
    #new_omega = np.array(omega)
    omega = np.zeros((Na,Nb,K,L))
    for link  in range(len(links_ratings)):
        i = links_array[link][0]
        j = links_array[link][1]
        if i==944:print(i,j)
        if j==944:print(link,i,j)
        rating = links_ratings[link]
        omega[i,j,:,:] = p_kl[:,:,rating]*(np.expand_dims(theta[i,:], axis=1)@np.expand_dims(eta[j,:],axis=0))
        suma = omega[i,j,:,:].sum()
        omega[i,j,:,:] /= suma+1e-16
    return omega


In [ ]:
omega_comp_arrays(len(ratings.nodes_a),len(ratings.nodes_b),ratings.pkl,ratings.nodes_a.theta,ratings.nodes_b.theta,ratings.nodes_a.K,ratings.nodes_b.K,
                               ratings.links_training,ratings.labels_training)

1436 15 944


IndexError: index 944 is out of bounds for axis 0 with size 943

In [ ]:
ratings.pkl[:,:,0]*(np.expand_dims(users.theta[15,:], axis=1)@np.expand_dims(movies.theta[944,:],axis=0))

IndexError: index 944 is out of bounds for axis 0 with size 943

In [ ]:
movies.theta.shape

(943, 10)

In [ ]:
ratings

(10, 10, 5)
(943, 10)
(1682, 10)


In [25]:
help(sbm.BiNet.__init__)

Help on function __init__ in module MMSBM_library:

__init__(self, links, links_label, *, nodes_a=None, nodes_b=None, Ka=1, nodes_a_name='nodes_a', Kb=1, nodes_b_name='nodes_b', separator='\t', dict_codes=None, dict_codes_a=None, dict_codes_b=None)
    Initialization of a BiNet class
    
    Parameters
    -----------
    links: str, pandas DataFrame
       DataFrame or directory containing the links between nodes_a and nodes_b and their labels.
    
    links_label: str
        Name of the column where the labels are stored in the links DataFrame.
    
    nodes_a: nodes_layer, str, DataFrame, None, default: None
        One of the nodes layer that forms the bipartite network
        - If nodes_layer: Existing instance of the nodes_layer class representing the first layer.
        - If str or pd.DataFrame: If str, a directory containing the file information about nodes_a.
        - If pd.DataFrame, DataFrame with nodes_a information.
        - If None: A simple nodes_layer will be cr